In [30]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt

In [43]:
train = pd.read_excel('/content/sales_train_final.xlsx',encoding='CP949')
test = pd.read_csv('/content/2020 june.csv',encoding="CP949")

In [ ]:
x = train[['노출(분)','상품군',	'판매단가',	'month',	'day',	'hour']]
y = train['취급액']
print(train.info())
print(x.shape,y.shape)

In [54]:
test2 = test[['노출(분)','상품군','판매단가','month','day','hour']]
category_num = {'가구':0, '가전':1, '건강기능':2, '농수축':3, '생활용품':4, '속옷':5, '의류':6, '이미용':7, '잡화':8, '주방':9, '침구':10}

In [55]:
for i,k in test2.iterrows():
  test2['상품군'].iloc[i] = category_num[test2['상품군'].iloc[i]]

In [63]:
test2.head()
test2 = test2.astype({'상품군':int})

In [64]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2716 entries, 0 to 2715
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   노출(분)   2716 non-null   float64
 1   상품군     2716 non-null   int64  
 2   판매단가    2716 non-null   int64  
 3   month   2716 non-null   int64  
 4   day     2716 non-null   int64  
 5   hour    2716 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 127.4 KB


In [32]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0)
print(x_train.shape,y_train.shape)

(23915, 6) (23915,)


In [33]:
from sklearn import preprocessing
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

import lightgbm as lgbm


import warnings
warnings.filterwarnings(action='ignore')

In [35]:
kfold = KFold(n_splits=10, random_state= 77, shuffle=True)

In [40]:
# LGBM Grid Search
params = {
    'task' : 'train',
    'objective' : 'regression',
    'subsample' : 0.8,
    'max_depth' : 7
}

param_grid = {
    'learning_rate' : [0.1],
    'feature_fraction' : [0.5,0.8],
    'num_leaves' : [31, 63, 127]
}

lgbm_model = lgbm.LGBMRegressor(**params, verbose=-1)
lgbm_grid = GridSearchCV(lgbm_model,
                         param_grid,
                         cv=kfold,
                         scoring='neg_mean_absolute_error',
                         return_train_score=True)

lgbm_grid.fit(x_train, y_train)

r2_score(lgbm_grid.predict(x_train),y_train)

lgbm_model.fit(x_train,y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=7,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=0.8,
              subsample_for_bin=200000, subsample_freq=0, task='train',
              verbose=-1)

In [66]:
predictions = lgbm_model.predict(test2)
print(predictions)

[14948319.1765886  14948319.1765886  24790187.34179095 ...
 21080243.86110543 11390447.66961174  9233009.35082535]


In [68]:
test['취급액'] = predictions

In [71]:
test.head(50)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,month,day,hour,minute,weekday,방송노출시간,방송날짜,holiday
0,2020-06-01 6:20,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,1.494832e+07,6,1,6,20,0,20.0,2020-06-01,0
1,2020-06-01 6:40,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,1.494832e+07,6,1,6,40,0,40.0,2020-06-01,0
2,2020-06-01 7:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,2.479019e+07,6,1,7,0,0,20.0,2020-06-01,0
3,2020-06-01 7:20,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,2.484235e+07,6,1,7,20,0,20.0,2020-06-01,0
4,2020-06-01 7:40,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,2.484235e+07,6,1,7,40,0,40.0,2020-06-01,0
5,2020-06-01 8:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,2.810696e+07,6,1,8,0,0,20.0,2020-06-01,0
6,2020-06-01 8:20,20.0,100381,201247,바비리스 퍼펙트 볼륨스타일러,이미용,59000,3.302372e+07,6,1,8,20,0,20.0,2020-06-01,0
7,2020-06-01 8:40,20.0,100381,201247,바비리스 퍼펙트 볼륨스타일러,이미용,59000,3.302372e+07,6,1,8,40,0,40.0,2020-06-01,0
8,2020-06-01 9:00,20.0,100381,201247,바비리스 퍼펙트 볼륨스타일러,이미용,59000,4.216568e+07,6,1,9,0,0,20.0,2020-06-01,0
9,2020-06-01 9:20,20.0,100638,201956,램프쿡 자동회전냄비,주방,109000,2.279419e+07,6,1,9,20,0,20.0,2020-06-01,0


In [70]:
test.to_csv('2020_june_sales_prediction.csv',encoding='cp949')